In [74]:
import glob

import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [75]:
data_base_dir = "data/by_date"

dates = []
dfs = {}
for fpath in glob.glob(data_base_dir + "/*"):
    df = pd.read_csv(fpath)
    date = df["date"].iloc[0]
    dfs[date] = df[df["question"].notnull()]
    dates.append(date)

dates.sort()

In [76]:
dfs[dates[0]]

,speaker,text,type,election_year,date,candidate,qud,question
6,John Kennedy,"Well, the Vice President and I came to the Con...",Pres,1960,1960-09-26,1,What are the key differences between the Democ...,"Senator, the Vice President in his campaign ha..."
11,Richard Nixon,It would be rather difficult to cover them in ...,Pres,1960,1960-09-26,1,What recommendations have been made after fore...,"Mr. Vice President, your campaign stresses the..."
17,John Kennedy,"Well, because I think that if the federal gove...",Pres,1960,1960-09-26,1,What are the potential consequences of withdra...,"Id like to ask this; Its a fact, I think, that..."
21,Richard Nixon,"Well, I would suggest, Mr. Vanocur, that uh – ...",Pres,1960,1960-09-26,1,What is the role of the President in decision-...,"Uh – Mr. Vice President, since the question of..."
26,John Kennedy,I didnt indicate. I did not advocate reducing ...,Pres,1960,1960-09-26,1,What is the speaker's perspective on addressin...,"Senator Kennedy, in connection with these prob..."
32,John Kennedy,"– reducing the interest rate. In my judgment, ...",Pres,1960,1960-09-26,1,What economic policies does the speaker propos...,"Senator, I believe in – in one of your speeche..."
37,Richard Nixon,Im awfully glad you ge- got that question beca...,Pres,1960,1960-09-26,1,What are the potential risks of federal involv...,Mr. Vice President you mentioned schools and i...
44,John Kennedy,"If I may take the bills, we did pass in the Se...",Pres,1960,1960-09-26,1,What challenges do legislative efforts face wh...,"Senator, youve been promising the voters that ..."
53,John Kennedy,"Well, I think theyre serious. I think its a ma...",Pres,1960,1960-09-26,1,What actions should be taken to ensure the Uni...,"Senator Kennedy, on another subject, Communism..."
58,Richard Nixon,Not at all. As a matter of fact your question ...,Pres,1960,1960-09-26,1,What factors contribute to America's economic ...,Mr. Vice President uh – in one of your earlier...


In [77]:
sum([len(df) for df in dfs.values()])

271

In [78]:
for i, d in enumerate(dates):
    print(f"{i}\t{d}\t{len(dfs[d])}\t{len(dfs[d].columns)}")

0	1960-09-26	10	8
1	1976-09-23	20	8
2	1980-09-21	12	8
3	1984-10-07	30	8
4	1988-09-25	23	8
5	1992-10-11	11	8
6	1996-10-06	21	8
7	2000-10-03	26	8
8	2004-09-30	18	8
9	2008-09-26	17	8
10	2012-10-03	25	8
11	2016-09-26	19	8
12	2020-09-29	39	8


In [79]:
dfs[dates[-1]]

,speaker,text,type,election_year,date,candidate,qud,question
5,Donald Trump,"Thank you very much, Chris. I will tell you ve...",Pres,2020,2020-09-29,1,What justification is provided for moving forw...,Our first subject is the Supreme Court. Presid...
9,Joe Biden,I - - the American people have a right to have...,Pres,2020,2020-09-29,1,Why should the American people have a say in t...,Our first subject is the Supreme Court. Presid...
84,Donald Trump,"All right. Well, first of all, I guess I'm deb...",Pres,2020,2020-09-29,1,What specific actions will be taken to reduce ...,"You, in the course of these four years have ne..."
155,Joe Biden,"Whatever the position I take on that, that wil...",Pres,2020,2020-09-29,1,What is the speaker encouraging the American p...,"So my question to you, is you have refused in ..."
176,Donald Trump,"If we would've listened to you, the country wo...",Pres,2020,2020-09-29,1,What are the consequences of not taking decisi...,"So my question to you, is you have refused in ..."
180,Donald Trump,You didn't think we should have closed our cou...,Pres,2020,2020-09-29,1,What is the argument being made about the effe...,"So my question to you, is you have refused in ..."
204,Donald Trump,"Well, I've spoken to the companies and we can ...",Pres,2020,2020-09-29,1,What obstacles are hindering the timely implem...,"Gentlemen - - gentlemen, let me move on to que..."
223,Joe Biden,No more than the question you just asked him. ...,Pres,2020,2020-09-29,1,What concerns are raised about the relationshi...,"I want to pick up on this question, though. Yo..."
250,Joe Biden,Because he doesn't have a plan. If I were runn...,Pres,2020,2020-09-29,1,What needs to be done to ensure businesses can...,When it comes to how the virus has been handle...
279,Donald Trump,"No, I think that masks are ok. You have to und...",Pres,2020,2020-09-29,1,What is the speaker's perspective on the use o...,"President Trump, you have begun to increasingl..."
